In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
df.loc[df["koi_disposition"]=="FALSE POSITIVE", "koi_disposition"] ="False_Positive"
df.koi_disposition

0            CONFIRMED
1       False_Positive
2       False_Positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_Positive
6987    False_Positive
6988         CANDIDATE
6989    False_Positive
6990    False_Positive
Name: koi_disposition, Length: 6991, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split
y=df["koi_disposition"]
X=df.drop(columns=["koi_disposition"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
print(X.shape, y.shape)

(6991, 40) (6991,)


In [5]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,...,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,...,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,...,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,...,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,...,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [8]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [9]:
y_train_categorical.shape

(5243, 3)

## Creating Deep Learning Model

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [14]:
# from keras.callbacks import EarlyStopping

callbacks=[EarlyStopping(monitor='val_loss', patience=2)]

model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 1s - loss: 0.5387 - accuracy: 0.7418
Epoch 2/60
164/164 - 0s - loss: 0.3728 - accuracy: 0.8047
Epoch 3/60
164/164 - 0s - loss: 0.3565 - accuracy: 0.8167
Epoch 4/60
164/164 - 0s - loss: 0.3501 - accuracy: 0.8255
Epoch 5/60
164/164 - 0s - loss: 0.3385 - accuracy: 0.8367
Epoch 6/60
164/164 - 0s - loss: 0.3398 - accuracy: 0.8371
Epoch 7/60
164/164 - 0s - loss: 0.3273 - accuracy: 0.8444
Epoch 8/60
164/164 - 0s - loss: 0.3230 - accuracy: 0.8451
Epoch 9/60
164/164 - 0s - loss: 0.3232 - accuracy: 0.8531
Epoch 10/60
164/164 - 0s - loss: 0.3178 - accuracy: 0.8543
Epoch 11/60
164/164 - 0s - loss: 0.3143 - accuracy: 0.8629
Epoch 12/60
164/164 - 0s - loss: 0.3138 - accuracy: 0.8543
Epoch 13/60
164/164 - 0s - loss: 0.3064 - accuracy: 0.8610
Epoch 14/60
164/164 - 0s - loss: 0.3054 - accuracy: 0.8644
Epoch 15/60
164/164 - 0s - loss: 0.3079 - accuracy: 0.8589
Epoch 16/60
164/164 - 0s - loss: 0.2974 - accuracy: 0.8678
Epoch 17/60
164/164 - 0s - loss: 0.2973 - accuracy: 0.8676
Epoch 

Epoch 45/60
164/164 - 0s - loss: 0.2531 - accuracy: 0.8909
Epoch 46/60
164/164 - 0s - loss: 0.2551 - accuracy: 0.8873
Epoch 47/60
164/164 - 0s - loss: 0.2497 - accuracy: 0.8899
Epoch 48/60
164/164 - 0s - loss: 0.2547 - accuracy: 0.8858
Epoch 49/60
164/164 - 0s - loss: 0.2526 - accuracy: 0.8869
Epoch 50/60
164/164 - 0s - loss: 0.2525 - accuracy: 0.8892
Epoch 51/60
164/164 - 0s - loss: 0.2498 - accuracy: 0.8905
Epoch 52/60
164/164 - 0s - loss: 0.2581 - accuracy: 0.8882
Epoch 53/60
164/164 - 0s - loss: 0.2510 - accuracy: 0.8850
Epoch 54/60
164/164 - 0s - loss: 0.2467 - accuracy: 0.8936
Epoch 55/60
164/164 - 0s - loss: 0.2437 - accuracy: 0.8915
Epoch 56/60
164/164 - 0s - loss: 0.2492 - accuracy: 0.8905
Epoch 57/60
164/164 - 0s - loss: 0.2469 - accuracy: 0.8934
Epoch 58/60
164/164 - 0s - loss: 0.2455 - accuracy: 0.8949
Epoch 59/60
164/164 - 0s - loss: 0.2484 - accuracy: 0.8915
Epoch 60/60
164/164 - 0s - loss: 0.2445 - accuracy: 0.8966


In [15]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2436 - accuracy: 0.8919
Normal Neural Network - Loss: 0.24359945952892303, Accuracy: 0.8918764591217041


## Make Predictions


In [16]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [17]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CANDIDATE' 'False_Positive' 'False_Positive' 'False_Positive'
 'CANDIDATE']
Actual Labels: ['CANDIDATE', 'False_Positive', 'False_Positive', 'False_Positive', 'CANDIDATE']


# Save the Model

In [18]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'deep_learning.sav'
joblib.dump(model, filename)

TypeError: cannot pickle 'weakref' object

In [19]:
# from keras.models import load_model

model.save('deep_learning.h5')